In [54]:
import torch
import numpy as np
from collections import defaultdict

import pandas as pd
import pathlib
from rich import print

import matplotlib.pyplot as plt

# set pyplot theme to ggplot2
plt.style.use("ggplot")
import wandb

In [55]:
source_csv_filepath = (
    "s01042024_df.csv"
)
source_csv_filepath = pathlib.Path(source_csv_filepath)

In [56]:
def load_data_as_df(filepath):
    df = pd.read_csv(filepath)

    # replace NA with 5
    df = df.fillna(5)
    # rename unnamed column to model_name
    df = df.rename(columns={"Unnamed: 0": "model_name"})
    # if a columne has testing/ensemble_3/ in it then rename it to replace testing/ensemble_3/ with ''
    df.columns = df.columns.str.replace("testing/ensemble_3/", "")
    # remove column exp_name
    # df = df.drop(columns=["exp_name"])
    return df

In [57]:
df = load_data_as_df(source_csv_filepath)


# visualize using tabulate for ipython noteboooks
from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

In [58]:
def move_column(df, column_name, position):
    """
    Move a specific column to a specified position in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.
    column_name (str): The name of the column to move.
    position (int): The position to move the column to.

    Returns:
    pandas.DataFrame: The modified DataFrame.
    """
    # Ensure the position is within the valid range
    if position < 0 or position > len(df.columns):
        raise ValueError(f"Position must be between 0 and {len(df.columns)}")

    # Get a list of the column names
    cols = df.columns.tolist()

    # Check if the column name is in the list
    if column_name in cols:
        # Remove the column from its current position
        cols.remove(column_name)

        # Insert the column at the specified position
        cols.insert(position, column_name)

        # Reorder the DataFrame
        df = df[cols]

    return df

df = move_column(df, 'imagenet1k.accuracy_top_1-epoch-mean', 1)  # Move 'hellobro' column to the second position
df = move_column(df, 'imagenet1k.accuracy_top_5-epoch-mean', 2)
df = move_column(df, 'imagenet1k.loss-epoch-mean', 3)

from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )


In [59]:
def rank_columns(df, keywords):
    """
    Rank the columns of a DataFrame based on keywords.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.
    keywords (dict): A dictionary mapping keywords to a boolean indicating whether to rank in
                     ascending order. For example, {'loss': True, 'acc': False}.

    Returns:
    pandas.DataFrame: The DataFrame with ranked columns.
    """
    for column in df.columns:
        for keyword, ascending in keywords.items():
            if keyword.lower() in column.lower():
               
                df[column] = df[column].rank(method='min', ascending=ascending)
                break  # Break the inner loop if keyword is found

    return df

# Define the keywords to look for in column names, and whether to rank in ascending order
keywords = {
    'loss': True,
    "mae": True,
    'acc': False,
    "miou": False,
    "auc": False,
    "bs": True,
    "aps": False,
}

df = rank_columns(df, keywords)
df = move_column(df, 'imagenet1k.accuracy_top_1-epoch-mean', 1)


In [60]:
from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

In [61]:
from scipy.stats import spearmanr

def reorder_columns_by_similarity(df):
    """
    Reorder the columns of a DataFrame based on their similarity to the second column,
    while keeping the first column in place.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.

    Returns:
    pandas.DataFrame: The DataFrame with reordered columns.
    """
    # Get the first and second column names
    first_column = df.columns[0]
    second_column = df.columns[3]
    
    # normalize the columns 1 onwards
    
    

    # Calculate the Spearman rank correlation between the second column and the others
    correlations = {column: np.abs(spearmanr(df[second_column], df[column]).correlation) for column in df.columns if column != first_column}

    # Sort the columns by their correlation to the second column in descending order
    sorted_columns = sorted(correlations, key=correlations.get, reverse=True)

    # Keep the first column in its place and reorder the rest
    df = df[[first_column] + sorted_columns]

    return df

df = reorder_columns_by_similarity(df)


In [62]:
# sort rows by the average rank of the columns except the first one
df["mean_rank"] = df.iloc[:, 1:].mean(axis=1)
df = df.sort_values("mean_rank")
df = df.drop(columns=["mean_rank"])

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

/var/folders/gh/_5t3nhn94t12m7rh483b8b7r0000gn/T/ipykernel_51283/2601485272.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [113]:
from typing import Optional
import plotly.graph_objects as go
import pandas as pd
import yaml  
    
def plot_heatmap(df: pd.DataFrame, task_type: Optional[str] = None):
    """
    Plot a heatmap of the DataFrame using Plotly, with rows sorted by their average rank.
    The x-axis labels reflect the order of the columns after they have been reordered by similarity.

    Parameters:
    df (pandas.DataFrame): The DataFrame to plot.

    Returns:
    None
    """
    # Define the task mapping
    
    if task_type:
        task_mapping = yaml.safe_load(open("task_mapping.yaml", "r"))
        column_names = task_mapping[task_type] + ["imagenet1k.top_accuracy_1"]
        keep_columns = [column for column in df.columns if column in column_names]
    else:
        keep_columns = df.columns[1:]
        
    # Exclude the label column from the data to be plotted
    labels = df[df.columns[0]]
    
    data = df[keep_columns]

    # Add a temporary column to the DataFrame with the average rank of each row
    data['average_rank'] = data.mean(axis=1)
    
    # Sort the DataFrame by this temporary column
    data.sort_values('average_rank', ascending=False, inplace=True)
    print(data['average_rank'])

    # Remove the temporary column
    data.drop('average_rank', axis=1, inplace=True)

    # Update the labels to match the new order
    labels = labels[data.index]
    labels = [label.replace("-224", "") for label in labels]
    data.columns = [column.replace("image", "img").replace("text", "txt").replace("_to_", "-").replace("accuracy_top_", "acc@").replace("accuracy", "acc").replace("-epoch-mean", "").replace("flickr30k", "f30k").replace("winoground", "wg").replace("pokemonblipcaptions", "pbc").replace("newyorkercaptioncontest", "nycc").replace("image-to-text", "img-to-txt").replace("text-to-image", "txt-to-img") for column in data.columns]
    # remove columns with terms "bs-1" and corresponding values
    # remove all columns matching any of the patterns in 'terms'
    terms = ["-bs", "-aps", "-auc", "focal", "nycc.loss", "ham10k.loss", "iwildcam.loss", "wg.loss", "clevr.loss",
    'clevr.loss_colour',
    'clevr.loss_count',
    'clevr.loss_material',
    'clevr.loss_shape',
    'clevr.loss_size',
    'clevr.loss_yes_no', 'happy.acc@5']
    columns = [column for column in data.columns if not any(term in column for term in terms)]
    # print(sorted(columns))
    data = data[columns]
    
    
    fig = go.Figure(data=go.Heatmap(
        z=data,
        x=columns,
        y=labels,
        colorscale='inferno_r',  # Use reversed Red color scale
    ))

    fig.update_layout(
        title='Model vs Dataset Rankings',
        xaxis_nticks=len(data.columns),
        yaxis_nticks=len(labels),
        autosize=True,
        width=1350,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )

    fig.show()
    fig.write_image("heatmap.png", scale=5)
df = move_column(df, 'imagenet1k.accuracy_top_1-epoch-mean', 1)  # Move 'hellobro' column to the second position
df = move_column(df, 'imagenet1k.accuracy_top_5-epoch-mean', 2)
df = move_column(df, 'imagenet1k.loss-epoch-mean', 3)
# apply logmax softmax to the columns in a columnwise manner


plot_heatmap(df)


/var/folders/gh/_5t3nhn94t12m7rh483b8b7r0000gn/T/ipykernel_51283/4215880335.py:32: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



14    14.095238
10    13.566138
11    13.010582
15    12.566138
13    11.174603
6     10.470899
4      9.142857
8      8.544974
1      7.037037
3      6.582011
5      5.883598
9      5.402116
2      4.936508
7      4.894180
12     4.470899
0      4.079365
Name: average_rank, dtype: float64